In [9]:
# 1. 필요한 라이브러리 설치 (가상 환경이 활성화된 상태에서)
!pip install langchain langchain-community langchain-core
!pip install transformers torch  # 또는 tensorflow (선호하는 딥러닝 프레임워크에 따라)
!pip install pytorch-lightning
!pip install datasets
!pip install scikit-learn
!pip install pandas
!pip install sentence-transformers


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import os
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import pandas as pd
import pytorch_lightning as pl
import torch
torch.set_float32_matmul_precision('medium')
import torch.nn as nn
import glob
from transformers import ElectraModel, AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split

In [2]:
###########################
# KOTE 파인튜닝 모델 로드 (체크포인트 파일 경로 수정)
###########################
best_ckpt_path_kote = './model/250127_KcElectra_kote.ckpt' # Colab 경로에 맞게 수정
print("Best checkpoint path (KOTE Finetuned):", best_ckpt_path_kote)

Best checkpoint path (KOTE Finetuned): ./model/250127_KcElectra_kote.ckpt


In [3]:
# KOTETagger 클래스는 이전 코드와 동일
class KOTETagger(pl.LightningModule): # KOTETagger 클래스 정의 (이전 코드에서 복사)
    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.electra = AutoModel.from_pretrained(MODEL_NAME, return_dict=True) # pretrained_electra 제거 및 직접 로드
        self.classifier = nn.Linear(self.electra.config.hidden_size, 44) # num_labels=44 (KOTE 라벨 개수)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)
        probs = torch.sigmoid(logits)
        loss = 0
        if labels is not None:
            loss = self.criterion(probs, labels)
        return loss, probs

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, _ = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return {"val_loss": loss}

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR, weight_decay=WEIGHT_DECAY) # 가중치 감쇠
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "step",
                "frequency": 1
            }
        }

In [4]:
###########################
# 토크나이저 로드
###########################
MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [5]:
emotion_labels = [
    "감동/감탄", "경악", "고마움", "공포/무서움", "귀찮음", "기대감", "기쁨", "깨달음",
    "놀람", "당황/난처", "부끄러움", "부담/안_내킴", "불쌍함/연민", "불안/걱정", "불평/불만",
    "비장함", "뿌듯함", "서러움", "슬픔", "신기함/관심", "아껴주는", "안심/신뢰", "안타까움/실망",
    "어이없음", "없음", "역겨움/징그러움", "우쭐댐/무시함", "의심/불신", "재미없음", "절망",
    "존경", "죄책감", "즐거움/신남", "증오/혐오", "지긋지긋", "짜증", "패배/자기혐오",
    "편안/쾌적", "한심함", "행복", "화남/분노", "환영/호의", "흐뭇함(귀여움/예쁨)", "힘듦/지침"
]

In [6]:
kote_finetuned_model = KOTETagger.load_from_checkpoint(best_ckpt_path_kote)
pretrained_electra = kote_finetuned_model.electra # 수정: electra backbone만 가져옴

In [7]:
###########################
# LightningModule 정의 (PoetryTagger) (기존 코드 활용 + 가중치 손실 함수, Dropout, Weight Decay, Learning Rate 감소, EarlyStopping patience 증가)
###########################
INITIAL_LR = 1e-5 # 학습률 감소 (원래 2e-5, 1e-5, 5e-6, 2e-6)
DROPOUT_RATE = 0.5 # Dropout 비율 (0.1, 0.3, 0.5) - Dropout 추가
WEIGHT_DECAY = 0.02 # Weight Decay 값 (0.001, 0.01, 0.02) - Weight Decay 추가
THRESHOLD = 0.3

class PoetryTagger(pl.LightningModule):
    def __init__(self, n_training_steps=None, n_warmup_steps=None, dropout_rate=DROPOUT_RATE): # dropout_rate hyperparameter
        super().__init__()
        self.electra = pretrained_electra # 수정: KOTE 파인튜닝 모델의 electra backbone 사용
        self.classifier = nn.Sequential( # nn.Sequential 사용하여 dropout layer 추가
            nn.Linear(self.electra.config.hidden_size, len(emotion_labels)),
            nn.Dropout(dropout_rate)
        ) # Classifier 출력층 크기 자동 조정
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss() # 기본 BCE Loss (가중치 미적용)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask)
        # [CLS] 토큰 기준으로 분류
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)
        probs = torch.sigmoid(logits)

        loss = 0
        if labels is not None:
            loss = self.criterion(probs, labels)

        return loss, probs

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, _ = self(input_ids, attention_mask, labels) # forward 함수에 weights 제거

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, outputs = self(input_ids, attention_mask, labels) # validation loss는 기존 BCE Loss 사용 (optional)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return {"val_loss": loss} # validation metrics are optional

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR, weight_decay=WEIGHT_DECAY) # 가중치 감쇠
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "step",
                "frequency": 1
            }
        }

In [8]:
# ###########################
# # Best checkpoint load & Evaluation (poetry-finetuning) (기존 코드 활용)
# ###########################
def get_latest_version_dir_poetry(base_dir="./lightning_logs/poetry-finetuning-3agreements-only"): # poetry-weighted-finetuning 로 변경
    # version_0, version_1, ... version_50 경로를 모두 찾아 리스트업
    version_dirs = glob.glob(os.path.join(base_dir, "version_*"))
    # 버전 숫자를 기준으로 정렬
    version_dirs.sort(key=lambda x: int(x.split("_")[-1]))
    if not version_dirs:
        raise FileNotFoundError(f"No version_* directories found under '{base_dir}'")
    # 가장 마지막(숫자가 가장 큰) 버전 경로 반환
    return version_dirs[-1]

def get_latest_checkpoint_poetry(version_dir):
    ckpt_dir = os.path.join(version_dir, "checkpoints")
    ckpt_list = glob.glob(os.path.join(ckpt_dir, "*.ckpt"))
    ckpt_list.sort()  # 파일명 기준 정렬
    if not ckpt_list:
        raise FileNotFoundError(f"No .ckpt found under '{ckpt_dir}'")
    # 가장 마지막 파일(정렬 기준)
    return ckpt_list[-1]

In [9]:
latest_version_dir_poetry = get_latest_version_dir_poetry("./lightning_logs/poetry-finetuning-3agreements-only") # poetry-weighted-finetuning 로 변경
best_ckpt_path_poetry = get_latest_checkpoint_poetry(latest_version_dir_poetry)
print("Best checkpoint path (Poetry Weighted Finetuned):", best_ckpt_path_poetry) # poetry-weighted-finetuning 로 변경

best_model_poetry = PoetryTagger.load_from_checkpoint(best_ckpt_path_poetry)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model_poetry.to(device)
best_model_poetry.eval()
best_model_poetry.freeze()

Best checkpoint path (Poetry Weighted Finetuned): ./lightning_logs/poetry-finetuning-3agreements-only/version_0/checkpoints/epoch9-val_loss0.2566.ckpt


In [10]:
sample_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까

웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.

도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.

오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.

문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.

그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을

In [11]:
encoding = tokenizer(
    sample_text,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    # 입력 텐서 또한 같은 device로 이동
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # forward
    _, predictions = best_model_poetry(input_ids, attention_mask)  # best_model_poetry 사용

# 추론 결과를 CPU로 가져와 numpy로 변환
predictions = predictions.flatten().cpu().numpy()

# 결과를 딕셔너리로 저장 (숫자값으로 변환)
result_dict = {
    label_name: float(round(score, 3))  # np.float32 -> float 변환
    for label_name, score in zip(emotion_labels, predictions)
    if score > THRESHOLD
}

# 결과 출력
print("\n[Sample Inference 결과]")
print(result_dict)

# 예시 출력
# {'불안/걱정': 0.336, '슬픔': 0.311}


[Sample Inference 결과]
{'공포/무서움': 0.4339999854564667, '놀람': 0.30799999833106995, '당황/난처': 0.5130000114440918, '부담/안_내킴': 0.39899998903274536, '불안/걱정': 0.5490000247955322, '불평/불만': 0.3019999861717224, '비장함': 0.33799999952316284, '서러움': 0.35600000619888306, '슬픔': 0.41600000858306885, '신기함/관심': 0.3089999854564667, '안타까움/실망': 0.30799999833106995, '의심/불신': 0.3050000071525574, '힘듦/지침': 0.34200000762939453}


### Ollama Llama + KPoEM classification Model LangChain

In [48]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [51]:
!pip install accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 3.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [54]:
!pip install --upgrade torch
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [58]:
!pip show torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: torch
Version: 2.7.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /Users/irolim/Documents/한중연/3차시/연구연계과제/langchain-llama-KPoEM/.venv/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, pytorch-lightning, sentence-transformers, torchaudio, torchmetrics, torchvision


In [60]:
model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드 (GPU가 없으면 device_map 제거)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,  # CPU 환경에서는 float32 사용
    device_map=None  # GPU가 없으면 auto 대신 None 사용
)

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.7,
    max_new_tokens=512
)

# 테스트 출력
output = pipe("안녕하세요, 오늘 기분은 어떤가요?")
print(output)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "안녕하세요, 오늘 기분은 어떤가요? - '어디서나' (2021) - 1회 - 1회\n'어디서나'는 2021년에 방영된 한국의 TV 드라마입니다. 주연은 이한na, 이하늘입니다. 이 드라마는 1회가 끝나고, 기분을 묻는 질문으로 시작됩니다. 이하늘 주연의 소설 '어디서나'를 바탕으로 한 이 드라마는, 한 여자의 일상과 그녀의 감정에 집중하는 작품입니다. 주인공은 한 여자가 다른 사람의 일상 속에서 발견된 자신의 모습을 통해 herself를 찾으려는 과정을 그립니다.\n1회는 주인공의 일상과 그녀의 감정에 집중하는 작품으로, 다양한 장면을 통해 그녀의 마음을 드러내고 있습니다. 이 드라마는, 주인공이 자신의 감정을 표현하는 과정에서 발견하는 새로운 것들을 보여주며, 독자와 시청자의 마음을 자극하는 작품입니다.\n'어디서나'는, 주인공이 자신의 감정을 표현하는 과정에서 발견하는 새로운 것들을 보여주는 작품입니다. 이 드라마는, 주인공이 자신의 감정을 표현하는 과정에서 발견하는 새로운 것을 통해 독자와 시청자의 마음을 자극하는 작품입니다. 이하늘 주연의 소설 '어디서나'를 바탕으로 한 이 드라마는, 한 여자의 일상과 그녀의 감정에 집중하는 작품입니다.\n1회는 주인공의 일상과 그녀의 감정에 집중하는 작품으로, 다양한 장면을 통해 그녀의 마음을 드러내고 있습니다. 이 드라마는, 주인공이 자신의 감정을 표현하는 과정에서 발견하는 새로운 것들을 보여주며, 독자와 시청자의 마음을 자극하는 작품입니다. 이하늘 주연의 소설 '어디서나'를 바탕으로 한 이 드라마는, 한 여자의 일상과 그녀의 감정에 집중하는 작품입니다.\n1회는 주인공의 일상과 그녀의 감정에 집중하는 작품으로, 다양한 장면을 통해 그녀의 마음을 드러내고 있습니다. 이 드라마는, 주인공이 자신의 감정을 표현하는 과정에서 발견하는 새로운 것들을 보여주며, 독자와 시청자의 마음을 자극하는 작품입니다. 이하늘 주연의 소설 '어디"}]


In [74]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 3072)

In [13]:
# 3. Ollama Llama 모델 로드
# Ollama 서버가 로컬에서 실행 중이어야 합니다.
# 모델 이름은 'ollama run llama3' 등으로 다운로드한 모델 이름과 일치해야 합니다.
# llm = Ollama(model="llama3.2")

/var/folders/cd/mj8pxhhd6ml7kpdhggf19l_80000gn/T/ipykernel_23423/1217402929.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [75]:
# 감정 분류 결과 예시 (임의의 함수 또는 모델로 대체)
def classify_emotion(sample_text):

    # 패딩 토큰 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # eos_token을 패딩 토큰으로 설정

    # 모델 임베딩 크기 조정
    model.resize_token_embeddings(len(tokenizer))

        
    # 예: {'슬픔': 0.6, '기쁨': 0.3, '분노': 0.1}
    encoding = tokenizer(
    sample_text,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
    )

    with torch.no_grad():
    # 입력 텐서 또한 같은 device로 이동
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)

    # forward
        _, predictions = best_model_poetry(input_ids, attention_mask)  # best_model_poetry 사용

# 추론 결과를 CPU로 가져와 numpy로 변환
    predictions = predictions.flatten().cpu().numpy()

# 결과를 딕셔너리로 저장 (숫자값으로 변환)
    result_dict = {
        label_name: float(round(score, 3))  # np.float32 -> float 변환
        for label_name, score in zip(emotion_labels, predictions)
        if score > THRESHOLD
    }

# 결과 출력
    print("\n[Sample Inference 결과]")
    print(result_dict)

# 예시 출력
# {'불안/걱정': 0.336, '슬픔': 0.311}
    return result_dict

In [68]:
# 2️⃣ 감정 기반 프롬프트 생성
def generate_prompt(emotion_scores):
    top_emotion = max(emotion_scores, key=emotion_scores.get)
    return f"""당신은 감정이 섬세한 한국 현대시 작가입니다. 
'{top_emotion}'의 감정을 중심으로 짧은 시를 한 편 창작해 주세요.""" 

In [76]:
# 4️⃣ LangChain 프롬프트 템플릿 구성
prompt = PromptTemplate.from_template("{emotion_prompt}")
chain = LLMChain(llm=llm, prompt=prompt)

In [77]:
# 5️⃣ 전체 흐름 함수
def emotion_to_poetry(user_input): #sample text
    emotion_scores = classify_emotion(user_input)
    emotion_prompt = generate_prompt(emotion_scores)
    poem = chain.run(emotion_prompt=emotion_prompt)
    print(emotion_scores) # 감정 점수 출력
    return poem

In [78]:
# 6️⃣ 테스트
input_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까

웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.

도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.

오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.

문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.

그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을
generated_poem = emotion_to_poetry(input_text)

print("🎴 생성된 시:\n")
print(generated_poem)

IndexError: index out of range in self

## 벡터 DB 연결

In [19]:
###########################
# 근현대시 데이터셋 로드 및 전처리 (기존 코드 활용 + 일치 라벨만 사용)
############################ 데이터프레임으로 불러오기 (실제 파일 경로로 수정)
df = pd.read_csv("../data/총합데이터셋_0601_5인 - 행단위.csv") # Colab 경로에 맞게 수정

# 감정 라벨 데이터를 리스트로 변환하는 함수
def labels_to_list(labels_str):
    if pd.isna(labels_str):
        return []
    return [label.strip() for label in labels_str.split(',')]

# 라벨 데이터를 리스트로 변환
df['annotator01_label_list'] = df['annotator01'].apply(labels_to_list)
df['annotator02_label_list'] = df['annotator02'].apply(labels_to_list)
df['annotator03_label_list'] = df['annotator03'].apply(labels_to_list)
df['annotator04_label_list'] = df['annotator04'].apply(labels_to_list)
df['annotator05_label_list'] = df['annotator05'].apply(labels_to_list)

In [20]:
def get_labels_agreed_by_at_least_k(row, k=3):
    """
    각 행(row)에 대해, 최소 k명 이상이 동의한 감정만 추출

    Parameters:
    - row: annotator label list들이 있는 DataFrame row
    - k: 동의한 annotator 최소 수 (기본 2명)

    Returns:
    - 감정 리스트 중 k명 이상이 공통으로 선택한 감정 리스트
    """
    all_labels = (
        row['annotator01_label_list'] +
        row['annotator02_label_list'] +
        row['annotator03_label_list'] +
        row['annotator04_label_list'] +
        row['annotator05_label_list']
    )
    counter = pd.Series(all_labels).value_counts() # 감정별 개수 세기
    return [label for label, count in counter.items() if count >= k] # k명 이상이 동의한 감정 리스트

In [21]:
def get_labels_agreed_by_at_least_k(row, k=3):
    """
    각 행(row)에 대해, 최소 k명 이상이 동의한 감정만 추출

    Parameters:
    - row: annotator label list들이 있는 DataFrame row
    - k: 동의한 annotator 최소 수 (기본 2명)

    Returns:
    - 감정 리스트 중 k명 이상이 공통으로 선택한 감정 리스트
    """
    all_labels = (
        row['annotator01_label_list'] +
        row['annotator02_label_list'] +
        row['annotator03_label_list'] +
        row['annotator04_label_list'] +
        row['annotator05_label_list']
    )
    counter = pd.Series(all_labels).value_counts() # 감정별 개수 세기
    return [label for label, count in counter.items() if count >= k] # k명 이상이 동의한 감정 리스트

In [22]:
# 2명 이상 동의한 감정 리스트로 새 컬럼 생성
df['common_labels'] = df.apply(lambda row: get_labels_agreed_by_at_least_k(row, k=3), axis=1)

df[['common_labels']].head(10) # 3명 이상 동의한 감정 리스트 확인

,common_labels
0,[비장함]
1,"[비장함, 부끄러움]"
2,[기대감]
3,"[패배/자기혐오, 절망, 슬픔, 힘듦/지침]"
4,"[기쁨, 기대감, 아껴주는]"
5,"[비장함, 불쌍함/연민, 아껴주는]"
6,[]
7,[비장함]
8,"[기대감, 감동/감탄]"
9,[슬픔]


In [23]:
# 일치하는 라벨만 있는 데이터만 필터링
df_agreement = df[df['common_labels'].map(len) > 0].reset_index(drop=True) # agreement 컬럼이 비어있지 않은 행만 선택

# 1차 데이터 csv 파일에서 'agreement' 컬럼이 비어 있지 않은 행만 선택
df_agreement = df[df['common_labels'].apply(lambda x: len(x) > 0)]

# 'agreement' 컬럼의 리스트들을 새로운 'labels' 컬럼에 할당
df_agreement['labels'] = df_agreement['common_labels']
df_agreement_reset = df_agreement.reset_index()

#  cleaned labels가 비어 있지 않은 행만 필터링 - Define df_agreement_cleaned FIRST
df_agreement_cleaned = df_agreement_reset[df_agreement_reset['labels'].map(len) > 0].reset_index(drop = True) # Line 46 (moved up) - df_agreement_cleaned is DEFINED here FIRST

# 불용 라벨 제거 (optional): ['nan', '', None] 라벨 제거
labels_to_remove = ['nan', '', None]


/var/folders/cd/mj8pxhhd6ml7kpdhggf19l_80000gn/T/ipykernel_23423/3686308332.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agreement['labels'] = df_agreement['common_labels']


In [24]:
def remove_labels(labels):
    return [label for label in labels if label not in labels_to_remove and pd.notna(label) and label != 'nan']

# Assign 'labels_cleaned' column to the ALREADY DEFINED df_agreement_cleaned
df_agreement_cleaned['labels_cleaned'] = df_agreement_reset['labels'].apply(remove_labels) # Line 43 (moved down) - Assign to df_agreement_cleaned AFTER it's defined

In [25]:
# 필터링된 데이터프레임
df_cleaned = df[df["common_labels"].apply(lambda x: len(x) > 0)].reset_index(drop=True)

In [26]:
df_cleaned.head(10)  # 필터링된 데이터프레임 확인

,lineID,poemID,본문,소제목,제목,저자,annotator01,annotator02,annotator03,annotator04,annotator05,annotator01_label_list,annotator02_label_list,annotator03_label_list,annotator04_label_list,annotator05_label_list,common_labels
0,1,1,죽는 날까지 하늘을 우러러,NaN,서시,윤동주,비장함,비장함,"뿌듯함, 비장함","비장함, 뿌듯함, 감동/감탄","비장함, 서러움, 슬픔",[비장함],[비장함],"[뿌듯함, 비장함]","[비장함, 뿌듯함, 감동/감탄]","[비장함, 서러움, 슬픔]",[비장함]
1,2,1,"한 점 부끄럼이 없기를,",NaN,서시,윤동주,"부끄러움, 비장함","부끄러움, 비장함, 기대감, 불안/걱정, 서러움, 슬픔","깨달음, 비장함, 뿌듯함","비장함, 부끄러움, 기대감",비장함,"[부끄러움, 비장함]","[부끄러움, 비장함, 기대감, 불안/걱정, 서러움, 슬픔]","[깨달음, 비장함, 뿌듯함]","[비장함, 부끄러움, 기대감]",[비장함],"[비장함, 부끄러움]"
2,3,1,잎새에 이는 바람에도,NaN,서시,윤동주,"기대감, 신기함/관심","기대감, 불안/걱정, 비장함","슬픔, 서러움, 불안/걱정, 당황/난처","비장함, 슬픔","감동/감탄, 신기함/관심, 편안/쾌적, 기대감","[기대감, 신기함/관심]","[기대감, 불안/걱정, 비장함]","[슬픔, 서러움, 불안/걱정, 당황/난처]","[비장함, 슬픔]","[감동/감탄, 신기함/관심, 편안/쾌적, 기대감]",[기대감]
3,4,1,나는 괴로워했다.,NaN,서시,윤동주,"절망, 슬픔, 패배/자기혐오","절망, 슬픔, 패배/자기혐오, 죄책감, 힘듦/지침, 비장함","당황/난처, 서러움, 죄책감, 패배/자기혐오","비장함, 슬픔, 패배/자기혐오, 절망, 힘듦/지침","슬픔, 서러움, 절망, 힘듦/지침, 패배/자기혐오","[절망, 슬픔, 패배/자기혐오]","[절망, 슬픔, 패배/자기혐오, 죄책감, 힘듦/지침, 비장함]","[당황/난처, 서러움, 죄책감, 패배/자기혐오]","[비장함, 슬픔, 패배/자기혐오, 절망, 힘듦/지침]","[슬픔, 서러움, 절망, 힘듦/지침, 패배/자기혐오]","[패배/자기혐오, 절망, 슬픔, 힘듦/지침]"
4,5,1,별을 노래하는 마음으로,NaN,서시,윤동주,"기쁨, 신기함/관심, 즐거움/신남, 흐뭇함(귀여움/예쁨), 뿌듯함","기쁨, 뿌듯함, 감동/감탄, 슬픔, 비장함, 아껴주는, 환영/호의, 기대감","고마움, 기대감, 기쁨, 아껴주는, 흐뭇함(귀여움/예쁨)","감동/감탄, 기대감, 기쁨, 아껴주는, 행복","즐거움/신남, 기대감, 기쁨, 행복","[기쁨, 신기함/관심, 즐거움/신남, 흐뭇함(귀여움/예쁨), 뿌듯함]","[기쁨, 뿌듯함, 감동/감탄, 슬픔, 비장함, 아껴주는, 환영/호의, 기대감]","[고마움, 기대감, 기쁨, 아껴주는, 흐뭇함(귀여움/예쁨)]","[감동/감탄, 기대감, 기쁨, 아껴주는, 행복]","[즐거움/신남, 기대감, 기쁨, 행복]","[기쁨, 기대감, 아껴주는]"
5,6,1,모든 죽어가는 것을 사랑해야지,NaN,서시,윤동주,"비장함, 불쌍함/연민, 아껴주는, 기쁨","비장함, 불쌍함/연민, 아껴주는, 슬픔, 환영/호의, 기대감, 안심/신뢰","불쌍함/연민, 비장함, 아껴주는, 존경","비장함, 슬픔","불쌍함/연민, 비장함","[비장함, 불쌍함/연민, 아껴주는, 기쁨]","[비장함, 불쌍함/연민, 아껴주는, 슬픔, 환영/호의, 기대감, 안심/신뢰]","[불쌍함/연민, 비장함, 아껴주는, 존경]","[비장함, 슬픔]","[불쌍함/연민, 비장함]","[비장함, 불쌍함/연민, 아껴주는]"
6,8,1,걸어가야겠다.,NaN,서시,윤동주,"비장함, 깨달음","비장함, 깨달음",없음,"기대감, 비장함","기대감, 비장함","[비장함, 깨달음]","[비장함, 깨달음]",[없음],"[기대감, 비장함]","[기대감, 비장함]",[비장함]
7,9,1,오늘 밤에도 별이 바람에 스치운다.,NaN,서시,윤동주,"흐뭇함(귀여움/예쁨), 기대감, 편안/쾌적","감동/감탄, 서러움, 슬픔, 아껴주는, 흐뭇함(귀여움/예쁨)","슬픔, 서러움, 불안/걱정","감동/감탄, 기대감, 기쁨, 신기함/관심, 아껴주는","감동/감탄, 기대감, 깨달음, 놀람, 신기함/관심","[흐뭇함(귀여움/예쁨), 기대감, 편안/쾌적]","[감동/감탄, 서러움, 슬픔, 아껴주는, 흐뭇함(귀여움/예쁨)]","[슬픔, 서러움, 불안/걱정]","[감동/감탄, 기대감, 기쁨, 신기함/관심, 아껴주는]","[감동/감탄, 기대감, 깨달음, 놀람, 신기함/관심]","[기대감, 감동/감탄]"
8,10,2,산모퉁이를 돌아 논가 외딴 우물을 홀로 찾아가선 가만히 들여다봅니다.,NaN,자화상,윤동주,"슬픔, 안타까움/실망","슬픔, 안타까움/실망, 서러움, 기대감, 불안/걱정, 비장함","패배/자기혐오, 한심함","힘듦/지침, 슬픔","기대감, 신기함/관심, 감동/감탄","[슬픔, 안타까움/실망]","[슬픔, 안타까움/실망, 서러움, 기대감, 불안/걱정, 비장함]","[패배/자기혐오, 한심함]","[힘듦/지침, 슬픔]","[기대감, 신기함/관심, 감동/감탄]",[슬픔]
9,11,2,우물 속에는 달이 밝고 구름이 흐르고 하늘이 펼치고 파아란 바람이 불고 가을이 있습니다.,NaN,자화상,윤동주,"흐뭇함(귀여움/예쁨), 환영/호의, 편안/쾌적, 신기함/관심","흐뭇함(귀여움/예쁨), 환영/호의, 편안/쾌적, 신기함/관심, 감동/감탄, 놀람, 서러움","감동/감탄, 신기함/관심","감동/감탄, 신기함/관심, 아껴주는, 편안/쾌적, 행복","감동/감탄, 기대감, 신기함/관심, 흐뭇함(귀여움/예쁨)","[흐뭇함(귀여움/예쁨), 환영/호의, 편안/쾌적, 신기함/관심]","[흐뭇함(귀여움/예쁨), 환영/호의, 편안/쾌적, 신기함/관심, 감동/감탄, 놀람,...","[감동/감탄, 신기함/관심]","[감동/감탄, 신기함/관심, 아껴주는, 편안/쾌적, 행복]","[감동/감탄, 기대감, 신기함/관심, 흐뭇함(귀여움/예쁨)]","[신기함/관심, 감동/감탄, 흐뭇함(귀여움/예쁨), 편안/쾌적]"


In [27]:
texts = df_cleaned["본문"].dropna().astype(str).tolist()
texts

['죽는 날까지 하늘을 우러러',
 '한 점 부끄럼이 없기를,',
 '잎새에 이는 바람에도',
 '나는 괴로워했다.',
 '별을 노래하는 마음으로',
 '모든 죽어가는 것을 사랑해야지',
 '걸어가야겠다.',
 '오늘 밤에도 별이 바람에 스치운다.',
 '산모퉁이를 돌아 논가 외딴 우물을 홀로 찾아가선 가만히 들여다봅니다.',
 '우물 속에는 달이 밝고 구름이 흐르고 하늘이 펼치고 파아란 바람이 불고 가을이 있습니다.',
 '그리고 한 사나이가 있습니다.',
 '어쩐지 그 사나이가 미워져 돌아갑니다.',
 '돌아가다 생각하니 그 사나이가 가엾어집니다.',
 '도로 가 들여다보니 사나이는 그대로 있습니다.',
 '다시 그 사나이가 미워져 돌아갑니다.',
 '돌아가다 생각하니 그 사나이가 그리워집니다.',
 '우물 속에는 달이 밝고 구름이 흐르고 하늘이 펼치고 파아란 바람이 불고 가을이 있고 추억처럼 사나이가 있습니다.',
 '지금 교회당 꼭대기',
 '십자가에 걸리었습니다.',
 '첨탑(尖塔)이 저렇게도 높은데',
 '어떻게 올라갈 수 있을까요.',
 '종소리도 들려 오지 않는데',
 '괴로웠던 사나이',
 '행복한 예수 그리스도에게',
 '처럼',
 '십자가가 허락된다면',
 '모가지를 드리우고',
 '꽃처럼 피어나는 피를',
 '어두워 가는 하늘 밑에',
 '조용히 흘리겠습니다.',
 '어디로 불려 가는 것일까',
 '바람이 부는데',
 '내 괴로움에는 이유가 없다.',
 '내 괴로움에는 이유가 없을까',
 '단 한 여자를 사랑한 일도 없다.',
 '시대를 슬퍼한 일도 없다.',
 '바람이 자꼬 부는데',
 '내 발이 반석 우에 섰다.',
 '내 발이 언덕 우에 섰다.',
 '고향에 돌아온 날 밤에',
 '내 백골(白骨)이 따라와 한방에 누웠다.',
 '어둔 방은 우주로 통하고',
 '하늘에선가 소리처럼 바람이 불어 온다.',
 '백골을 들여다보며',
 '눈물짓는 것이 내가 우는 것이냐',
 '백골이 우는 것이냐',
 '아름다운 혼이 우는 것이냐',
 '지조

In [28]:
from langchain.docstore.document import Document

In [29]:
# 2️⃣ 문장들을 Document 형태로 변환
documents = [Document(page_content=text) for text in texts]

In [30]:
from typing import List
from dataclasses import dataclass

@dataclass
class Document:
    metadata: dict
    page_content: str

def add_common_labels_to_documents(documents: List[Document], df_cleaned, column_name="common_labels"):
    """
    documents의 metadata 딕셔너리에 df_cleaned의 'common_labels' 열 값을 추가하는 함수.
    
    Args:
        documents (List[Document]): Document 객체 리스트.
        df_cleaned (pd.DataFrame): 'common_labels' 열을 포함하는 데이터프레임.
        column_name (str): 추가할 열 이름. 기본값은 'common_labels'.
    
    Returns:
        List[Document]: metadata가 업데이트된 Document 객체 리스트.
    """
    for i, doc in enumerate(documents):
        if i < len(df_cleaned):
            # df_cleaned의 'common_labels' 값을 metadata에 추가
            doc.metadata[column_name] = df_cleaned[column_name].iloc[i]
        else:
            # df_cleaned에 없는 경우 빈 리스트 추가
            doc.metadata[column_name] = []
    return documents

In [31]:
# 함수 호출
updated_documents = add_common_labels_to_documents(documents, df_cleaned)

# 결과 출력
for doc in updated_documents:
    print(doc)

page_content='죽는 날까지 하늘을 우러러' metadata={'common_labels': ['비장함']}
page_content='한 점 부끄럼이 없기를,' metadata={'common_labels': ['비장함', '부끄러움']}
page_content='잎새에 이는 바람에도' metadata={'common_labels': ['기대감']}
page_content='나는 괴로워했다.' metadata={'common_labels': ['패배/자기혐오', '절망', '슬픔', '힘듦/지침']}
page_content='별을 노래하는 마음으로' metadata={'common_labels': ['기쁨', '기대감', '아껴주는']}
page_content='모든 죽어가는 것을 사랑해야지' metadata={'common_labels': ['비장함', '불쌍함/연민', '아껴주는']}
page_content='걸어가야겠다.' metadata={'common_labels': ['비장함']}
page_content='오늘 밤에도 별이 바람에 스치운다.' metadata={'common_labels': ['기대감', '감동/감탄']}
page_content='산모퉁이를 돌아 논가 외딴 우물을 홀로 찾아가선 가만히 들여다봅니다.' metadata={'common_labels': ['슬픔']}
page_content='우물 속에는 달이 밝고 구름이 흐르고 하늘이 펼치고 파아란 바람이 불고 가을이 있습니다.' metadata={'common_labels': ['신기함/관심', '감동/감탄', '흐뭇함(귀여움/예쁨)', '편안/쾌적']}
page_content='그리고 한 사나이가 있습니다.' metadata={'common_labels': ['비장함']}
page_content='어쩐지 그 사나이가 미워져 돌아갑니다.' metadata={'common_labels': ['불쌍함/연민', '안타까움/실망']}
page_content='돌아가다 생각하니 그 사나

In [32]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [33]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from langchain.embeddings.base import Embeddings

class KcELECTRAEmbeddings(Embeddings):
    def __init__(self, model_name: str = "beomi/KcELECTRA-base", device: str = "cpu"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.device = device

    def _embed(self, text: str):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :]
        return cls_embedding.squeeze().cpu().numpy()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return [self._embed(text).tolist() for text in texts]

    def embed_query(self, text: str) -> list[float]:
        return self._embed(text).tolist()


In [34]:

# 3️⃣ 벡터 임베딩 모델 로딩 (한국어 지원하는 모델 권장) KcElectra -> backbone 모델로 사용
embedding_model = KcELECTRAEmbeddings()
# embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

In [41]:
! pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [35]:
# 4️⃣ FAISS VectorDB 생성
vectorstore = FAISS.from_documents(documents, embedding_model)

In [44]:
# 2️⃣ 감정 기반 프롬프트 생성
def generate_prompt_withVector(emotion_scores, context_snippets):
    top_emotion = max(emotion_scores, key=emotion_scores.get)
    return f"""당신은 감정이 섬세한 한국 근현대 시인입니다.
'{top_emotion}'의 감정을 중심으로 한국어로 짧은 시를 한 편 창작해 주세요.
다음은 감정의 분위기를 도와줄 참고 문장입니다:
{context_snippets} 이 문장들은 5명의 어노테이터가 44개 감정 라벨 중 최대 10개를 선택한 결과에서 3인 이상 공통된 감정만 추출한 신뢰도 높은 데이터셋입니다.
이 문장들의 정서와 단어, 옛스러운 한국 고유의 표현을 사용하여 시를 지어 주세요.
"""

In [47]:
# 8️⃣ 전체 체인
def emotion_to_poetry(user_input):
    scores = classify_emotion(user_input)
    top_emotion = max(scores, key=scores.get)

    # 관련 시구 검색
    context_snippets = vectorstore.similarity_search(top_emotion, k=10)  # k=100로 설정, 필요에 따라 조정 가능

    # 프롬프트 생성
    full_prompt = generate_prompt_withVector(scores, context_snippets)

    # LangChain Prompt + LLM 실행
    prompt = PromptTemplate.from_template("{emotion_prompt}")
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(emotion_prompt=full_prompt)
    print(context_snippets)

    return result

In [46]:
# 9️⃣ 테스트 실행
user_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까


웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.


도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.


오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.


문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.


그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을
generated_poem = emotion_to_poetry(user_text)

print("🎴 생성된 시:\n", generated_poem)


[Sample Inference 결과]
{'공포/무서움': 0.4350000023841858, '놀람': 0.30799999833106995, '당황/난처': 0.5139999985694885, '부담/안_내킴': 0.39899998903274536, '불안/걱정': 0.5479999780654907, '불평/불만': 0.30300000309944153, '비장함': 0.33899998664855957, '서러움': 0.35100001096725464, '슬픔': 0.414000004529953, '신기함/관심': 0.3109999895095825, '안타까움/실망': 0.30799999833106995, '의심/불신': 0.30799999833106995, '힘듦/지침': 0.33899998664855957}
[Document(id='c937a09c-dc53-41d2-834d-685c8bbfe9c6', metadata={'common_labels': ['안타까움/실망']}, page_content='이밤이라도'), Document(id='dcd63fb7-88bc-4cb8-8ea7-45b82f99dd1e', metadata={'common_labels': ['기대감', '아껴주는', '환영/호의', '기쁨']}, page_content='당신님의 편지를'), Document(id='4ff70e58-d6f7-42a0-8ee8-b0ef848f345d', metadata={'common_labels': ['슬픔', '절망']}, page_content='꿈은 깨어졌다'), Document(id='0c9f35b0-59ec-4f16-ad55-44bbfa42a1cd', metadata={'common_labels': ['편안/쾌적']}, page_content='바람자는 이 저녁'), Document(id='03bcb272-1967-4172-90e3-2f0309c93bb1', metadata={'common_labels': ['신기함/관심', '불안/걱정']}, pag